In [2]:
#换页
import dash
import dash_html_components as html
import dash_bootstrap_components as dbc

from dash.dependencies import Input, Output
import dash_core_components as dcc 

# from pandas_datareader import data as web 
import yfinance as yf
from datetime import datetime as dt

import numpy as np 
import pandas as pd
from ta.trend import MACD
from ta.momentum import StochasticOscillator
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# app = dash.Dash(__name__)

app.layout = html.Div(
    dbc.Container(
        dbc.Tabs(
            [
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('The latest international macro-enocomic data are shown below'),
                        dcc.Graph(id='page1-graph1'),
                        dcc.Graph(id='page1-graph2'),
                        dcc.Graph(id='page1-graph3'),
                        dcc.Graph(id='page1-graph4'),
                        dcc.Graph(id='page1-graph5'),
                        dcc.Graph(id='page1-graph6'),
                        dcc.Graph(id='page1-graph7'),
                        dcc.Graph(id='page1-graph8'),
                        dcc.Graph(id='page1-graph9')
                    ],
                    label='Financial Data Dashboard'
                ),
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('Choose the interested stock in the dropdown'),
                        dcc.Dropdown(
                        id='page2-dropdown',
                        options=[
                            {'label': 'Coke', 'value': 'COKE'},
                            {'label': 'Tesla', 'value': 'TSLA'},
                            {'label': 'Apple', 'value': 'AAPL'},
                            {'label': 'Amazon', 'value': 'AMZN'},
                            {'label': 'S&P 500', 'value': '^GSPC'}
                        ],value='TSLA'),  
                        dcc.Graph(id='page2-graph1'),
                        dcc.Graph(id='page2-graph2'),
                        dcc.Graph(id='page2-graph3'),
                        dcc.Graph(id='page2-graph4'),
                        dcc.Graph(id='page2-graph5'),
                        dcc.Graph(id='page2-graph6')
                    ],
                    label='Stock Data Visualization'
                ),
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('这是选项卡3')
                    ],
                    label='选项卡3'
                ),
            ]
        ),
        style={'margin-top': '50px'}
    )
)
@app.callback([Output('page2-graph1','figure'),Output('page2-graph2','figure'),Output('page2-graph3','figure'),Output('page2-graph4','figure'),Output('page2-graph5','figure'),Output('page2-graph6','figure')], [Input('page2-dropdown', 'value')])
def update_graph(selected_dropdown_value):

    df =  yf.download(selected_dropdown_value, start='2020-01-01', proxy="127.0.0.1:33210")
    
    fig1 = go.Figure(go.Candlestick(x=df.index,
      open=df['Open'],
      high=df['High'],
      low=df['Low'],
      close=df['Close'],
      showlegend=False),
      layout=go.Layout(title=go.layout.Title(text=f"Stock Price and Moving Average of {selected_dropdown_value}")))

    #把非交易日排除掉
    # hide weekends
    fig1.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])]) 
    # removing all empty dates
    # build complete timeline from start date to end date
    dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
    # retrieve the dates that ARE in the original datset
    dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
    # define dates with missing values
    dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
    fig1.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

    #add moving average
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    fig1.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='MA 5'))

    fig1.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='MA 20'))
    fig1.add_trace(go.Scatter(x=df.index, 
                         y=df['MA50'], 
                         opacity=0.7, 
                         line=dict(color='pink', width=2), 
                         name='MA 50'))

    # Plot volume trace on 2nd row 
    fig2 = go.Figure(go.Bar(x=df.index, 
                     y=df['Volume'],
                    ),layout=go.Layout(title=go.layout.Title(text=f"Stock Trading Volumn of {selected_dropdown_value}")))
    # Plot MACD trace on 3rd row
    # MACD
    macd = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)
    fig3 = go.Figure(go.Bar(x=df.index, 
                     y=macd.macd_diff(),
                     name = 'MACD_DIF',
                    ),layout=go.Layout(title=go.layout.Title(text=f"MACD Indicator {selected_dropdown_value}")))
    fig3.add_trace(go.Scatter(x=df.index,
                         y=macd.macd(),
                         line=dict(color='black', width=2),
                         name = 'MACD_DEA'
                        ))
    fig3.add_trace(go.Scatter(x=df.index,
                         y=macd.macd_signal(),
                         line=dict(color='blue', width=1),
                         name = 'MACD moving average of 3 days'
                        ))
    # Plot stochastics trace on 4th row
    # stochastic
    stoch = StochasticOscillator(high=df['High'],
                             close=df['Close'],
                             low=df['Low'],
                             window=14, 
                             smooth_window=3)
    fig4 = go.Figure(go.Scatter(x=df.index,
                         y=stoch.stoch(),
                         line=dict(color='black', width=2),
                         name = 'stochastic'
                        ),layout=go.Layout(title=go.layout.Title(text=f"Stochastic of {selected_dropdown_value}")))
    fig4.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch_signal(),
                         name = 'stochastic moving average of 3 days',
                         line=dict(color='blue', width=1),
                        ))
    #传统的stochastics oscillator 80超买，20超卖,把这个加进stochastics
    
    const1 = np.linspace(start=20, stop=20,num=len(df.index))
    const2 = np.linspace(start=80, stop=80,num=len(df.index))
    fig4.add_trace(go.Scatter(x=df.index,
                         y=const1,
                         line=dict(color='red',width=1),
                         name = '20'
                        ))
    fig4.add_trace(go.Scatter(x=df.index,
                         y=const2,
                         line=dict(color='green',width=1),
                         name = '80'
                        ))
    
    #fig5 volatility
    df.insert(df.shape[1],'Ln',0)
    df.insert(df.shape[1],'Ln2',0)

    for i in range(df.shape[0]-1):
        df['Ln'].iloc[i+1] = math.log(df['Close'].iloc[i+1]/df['Close'].iloc[i])
        df['Ln2'].iloc[i+1] = df['Ln'].iloc[i+1]**2
    df.insert(df.shape[1],'Volatility',0)
    for i in range(df.shape[0]-2):
        df['Volatility'].iloc[i+2] = math.sqrt(df['Ln2'].iloc[1:i+2].sum()/(i+2-1))
    fig5 = go.Figure(go.Scatter(x=df.index,
                         y=df['Volatility'],
                         line=dict(color='blue',width=1)
                        ),layout=go.Layout(title=go.layout.Title(text=f"Volatility of {selected_dropdown_value}")))
    
    #figure6 value at risk
    df['daily_ret'] = df['Close'].pct_change(1)
    # print(df)
    sRate = df['daily_ret'].iloc[1:].sort_values(ascending=True)
    # print(sRate)
    p = np.percentile(sRate, (1, 5, 10), interpolation='midpoint') # 输出分位度为1%，5%和10%即置信度99%，95%和90%时的值
    print(p) #1%分位值为第一个-0.05377872，即根据历史数据，value at risk回报率高于-0.05377872的可能为99%
    fig6 = go.Figure(go.Bar(x=['1%','5%','10%'],
                     y=p
                     ),layout=go.Layout(title=go.layout.Title(text=f"Value at risk on 1%, 5% and 10% of {selected_dropdown_value}")))
    return fig1, fig2, fig3, fig4, fig5, fig6

# page1
@app.callback([Output('page1-graph1','figure'),Output('page1-graph2','figure'),Output('page1-graph3','figure'),Output('page1-graph4','figure'),Output('page1-graph5','figure'),Output('page2-graph6','figure'),Output('page1-graph7','figure'),Output('page1-graph8','figure'),Output('page2-graph9','figure')], [Input('page2-dropdown', 'value')])
def update_graph_page1():
    #show gdp data
    world_gdp_data = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4251000.csv')
    # world_gdp_data = world_gdp_data.reset_index(drop=True)
    world_gdp = world_gdp_data.iloc[259][4:66]
    China_gdp = world_gdp_data.iloc[40][4:66]
    USA_gdp = world_gdp_data.iloc[251][4:66]
    UK_gdp = world_gdp_data.iloc[81][4:66]
        # print(Russian_gdp)
    x = np.arange(1960, 2022, 1)
    fig1 = go.Figure(go.Scatter(
                    x = x,
                    y = world_gdp, 
                    name = 'World GDP data'),layout=go.Layout(title=go.layout.Title(text="GDP data of world main economy"),
                                                             xaxis_title="Year", yaxis_title="GDP/million dollars",)
                )
    fig1.add_trace(go.Scatter(
                    x = x,
                    y = China_gdp, 
                    name = 'China GDP data')
                )
    fig1.add_trace(go.Scatter(
                    x = x,
                    y = USA_gdp, 
                    name = 'USA GDP data')
                )
    fig1.add_trace(go.Scatter(
                    x = x,
                    y = UK_gdp, 
                    name = 'UK GDP data')
                )

    #show gdp per capital data
    world_gdp_perCapital_data = pd.read_csv('API_NY.GDP.PCAP.CD_DS2_en_csv_v2_4251004.csv')
    world_gdp_perCapital = world_gdp_perCapital_data.iloc[259][4:66]
    China_gdp_perCapital = world_gdp_perCapital_data.iloc[40][4:66]
    USA_gdp_perCapital = world_gdp_perCapital_data.iloc[251][4:66]
    UK_gdp_perCapital = world_gdp_perCapital_data.iloc[81][4:66]
    x2 = np.arange(1960, 2022, 1)
    fig2 = go.Figure(go.Scatter(
                    x = x2,
                    y = world_gdp_perCapital, 
                    name = 'World GDP per capital data'),layout=go.Layout(title=go.layout.Title(text="GDP per capital data of world main economy"),
                                                             xaxis_title="Year", yaxis_title='GDP per capital/dollars',)
                )
    fig2.add_trace(go.Scatter(
                    x = x2,
                    y = China_gdp_perCapital, 
                    name = 'China GDP per capital data')
                )
    fig2.add_trace(go.Scatter(
                    x = x2,
                    y = USA_gdp_perCapital, 
                    name = 'USA GDP per capital data')
                )
    fig2.add_trace(go.Scatter(
                    x = x2,
                    y = UK_gdp_perCapital, 
                    name = 'UK GDP per capitaldata')
                )

    #show gdp growth rate data
    world_gdp_growth_data = pd.read_csv('API_NY.GDP.PCAP.KD.ZG_DS2_en_csv_v2_4250851.csv')
    world_gdp_growth = world_gdp_growth_data.iloc[259][4:66]
    China_gdp_growth = world_gdp_growth_data.iloc[40][4:66]
    USA_gdp_growth = world_gdp_growth_data.iloc[251][4:66]
    UK_gdp_growth = world_gdp_growth_data.iloc[81][4:66]
    x3 = np.arange(1960, 2022, 1)
    fig3 = go.Figure(go.Scatter(
                    x = x3,
                    y = world_gdp_growth, 
                    name = 'World GDP growth rate'),layout=go.Layout(title=go.layout.Title(text="GDP growth rate of world main economy"),
                                                             xaxis_title="Year", yaxis_title='GDP growth rate/percent',)
                )
    fig3.add_trace(go.Scatter(
                    x = x3,
                    y = China_gdp_growth, 
                    name = 'China GDP growth rate')
                )
    fig3.add_trace(go.Scatter(
                    x = x3,
                    y = USA_gdp_growth, 
                    name = 'USA GDP growth rate')
                )
    fig3.add_trace(go.Scatter(
                    x = x3,
                    y = UK_gdp_growth, 
                    name = 'UK GDP growth rate')
                )

    # show world electricity consumption data
    world_electricity_data = pd.read_csv('API_EG.USE.ELEC.KH.PC_DS2_en_csv_v2_4251643.csv')
    world_electricity = world_electricity_data.iloc[259][4:66]
    China_electricity = world_electricity_data.iloc[40][4:66]
    USA_electricity = world_electricity_data.iloc[251][4:66]
    UK_electricity = world_electricity_data.iloc[81][4:66]
    x4 = np.arange(1960, 2022, 1)
    fig4 = go.Figure(go.Scatter(
                    x = x4,
                    y = world_electricity, 
                    name = 'World electricity consumption'),layout=go.Layout(title=go.layout.Title(text="Electricity Consumption of world main economy"),
                                                             xaxis_title="Year", yaxis_title='Electricity consumption / kW·h per capital',)
                )
    fig4.add_trace(go.Scatter(
                    x = x4,
                    y = China_electricity, 
                    name = 'China electricity consumption')
                )
    fig4.add_trace(go.Scatter(
                    x = x4,
                    y = USA_electricity, 
                    name = 'USA electricity consumption')
                )
    fig4.add_trace(go.Scatter(
                    x = x4,
                    y = UK_electricity, 
                    name = 'UK electricity consumption')
                )

    # show world population rate data
    world_population_data = pd.read_csv('API_SP.POP.GROW_DS2_en_csv_v2_4251293.csv')
    world_population = world_population_data.iloc[259][4:66]
    China_population = world_population_data.iloc[40][4:66]
    USA_population = world_population_data.iloc[251][4:66]
    UK_population = world_population_data.iloc[81][4:66]
    x5 = np.arange(1960, 2022, 1)
    fig5 = go.Figure(go.Scatter(
                    x = x5,
                    y = world_population, 
                    name = 'World population growth rate'),layout=go.Layout(title=go.layout.Title(text="Population growth rate of world main economy"),
                                                             xaxis_title="Year", yaxis_title='Population growth rate / percent',)
                )
    fig5.add_trace(go.Scatter(
                    x = x5,
                    y = China_population, 
                    name = 'China population growth rate')
                )
    fig5.add_trace(go.Scatter(
                    x = x5,
                    y = USA_population, 
                    name = 'USA population growth rate')
                )
    fig5.add_trace(go.Scatter(
                    x = x5,
                    y = UK_population, 
                    name = 'UK population growth rate')
                )

    # show world inflation data(GDP deflator)
    world_inflation_data = pd.read_csv('API_NY.GDP.DEFL.KD.ZG_DS2_en_csv_v2_4250766.csv')
    world_inflation = world_inflation_data.iloc[259][4:66]
    China_inflation = world_inflation_data.iloc[40][4:66]
    USA_inflation = world_inflation_data.iloc[251][4:66]
    UK_inflation = world_inflation_data.iloc[81][4:66]
    x6 = np.arange(1960, 2022, 1)
    fig6 = go.Figure(go.Scatter(
                    x = x6,
                    y = world_inflation, 
                    name = 'World inflation rate'),layout=go.Layout(title=go.layout.Title(text="inflation rate of world main economy"),
                                                             xaxis_title="Year", yaxis_title='inflation rate / percent',)
                )
    fig6.add_trace(go.Scatter(
                    x = x6,
                    y = China_inflation, 
                    name = 'China inflation rate')
                )
    fig6.add_trace(go.Scatter(
                    x = x6,
                    y = USA_inflation, 
                    name = 'USA inflation rate')
                )
    fig6.add_trace(go.Scatter(
                    x = x6,
                    y = UK_inflation, 
                    name = 'UK inflation rate')
                )

    #show gold data
    start_date = '2021-1-1'
    today = datetime.date.today()
    latest_day = today + datetime.timedelta(days=-1)
    # end_date = '2022-7-22'
    Gold_data = yf.download('GLD', start=start_date, end=latest_day, group_by="ticker", proxy="127.0.0.1:33210",
                                index_col=0, auto_adjust=True)
    fig7 = go.Figure(go.Scatter(Gold_data['Close']),layout=go.Layout(title=go.layout.Title(text=f'Gold Price from{start_date} until {today}'),
                                                             xaxis_title="Year", yaxis_title='Gold Price/dollar',)
                )

    #show oil data
    Oil_data = yf.download('USO', start=start_date, end=today, group_by="ticker", proxy="127.0.0.1:33210",
                               index_col=0, auto_adjust=True)
    fig8 = go.Figure(go.Scatter(Oil_data['Close']),layout=go.Layout(title=go.layout.Title(text=f'Oil Price from{start_date} until {today}'),
                                                             xaxis_title="Year", yaxis_title='Oil Price/dollar',)
                )

    #show gas data
    Gas_data = yf.download('UNG', start=start_date, end=today, group_by="ticker", proxy="127.0.0.1:33210",
                               index_col=0, auto_adjust=True)
    fig9 = go.Figure(go.Scatter(asOil_data['Close']),layout=go.Layout(title=go.layout.Title(text=f'Gas Price from{start_date} until {today}'),
                                                             xaxis_title="Year", yaxis_title='Gas Price/dollar',)
                )
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9 


if __name__ == '__main__':
    app.run_server(port=8051)
# if __name__ == '__main__':
#     app.run_server(debug=True)

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051 (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2022 14:45:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 14:45:11] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 14:45:11] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 14:45:11] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Jul/2022 14:45:11] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Jul/2022 14:45:11] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


In [4]:
# plot of page1
import plotly.graph_objects as go
 #show gdp data
world_gdp_data = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4251000.csv')
# world_gdp_data = world_gdp_data.reset_index(drop=True)
world_gdp = world_gdp_data.iloc[259][4:66]
China_gdp = world_gdp_data.iloc[40][4:66]
USA_gdp = world_gdp_data.iloc[251][4:66]
UK_gdp = world_gdp_data.iloc[81][4:66]
        # print(Russian_gdp)
x = np.arange(1960, 2022, 1)
fig1 = go.Figure(go.Scatter(
                    x = x,
                    y = world_gdp, 
                    name = 'World GDP data'),layout=go.Layout(title=go.layout.Title(text="GDP data of world main economy"),
                                                             xaxis_title="Year", yaxis_title="GDP/million dollars",)
                )
fig1.add_trace(go.Scatter(
                    x = x,
                    y = China_gdp, 
                    name = 'China GDP data')
                )
fig1.add_trace(go.Scatter(
                    x = x,
                    y = USA_gdp, 
                    name = 'USA GDP data')
                )
fig1.add_trace(go.Scatter(
                    x = x,
                    y = UK_gdp, 
                    name = 'UK GDP data')
                )

#show gdp per capital data
world_gdp_perCapital_data = pd.read_csv('API_NY.GDP.PCAP.CD_DS2_en_csv_v2_4251004.csv')
world_gdp_perCapital = world_gdp_perCapital_data.iloc[259][4:66]
China_gdp_perCapital = world_gdp_perCapital_data.iloc[40][4:66]
USA_gdp_perCapital = world_gdp_perCapital_data.iloc[251][4:66]
UK_gdp_perCapital = world_gdp_perCapital_data.iloc[81][4:66]
x2 = np.arange(1960, 2022, 1)
fig2 = go.Figure(go.Scatter(
                    x = x2,
                    y = world_gdp_perCapital, 
                    name = 'World GDP per capital data'),layout=go.Layout(title=go.layout.Title(text="GDP per capital data of world main economy"),
                                                             xaxis_title="Year", yaxis_title='GDP per capital/dollars',)
                )
fig2.add_trace(go.Scatter(
                    x = x2,
                    y = China_gdp_perCapital, 
                    name = 'China GDP per capital data')
                )
fig2.add_trace(go.Scatter(
                    x = x2,
                    y = USA_gdp_perCapital, 
                    name = 'USA GDP per capital data')
                )
fig2.add_trace(go.Scatter(
                    x = x2,
                    y = UK_gdp_perCapital, 
                    name = 'UK GDP per capitaldata')
                )

#show gdp growth rate data
world_gdp_growth_data = pd.read_csv('API_NY.GDP.PCAP.KD.ZG_DS2_en_csv_v2_4250851.csv')
world_gdp_growth = world_gdp_growth_data.iloc[259][4:66]
China_gdp_growth = world_gdp_growth_data.iloc[40][4:66]
USA_gdp_growth = world_gdp_growth_data.iloc[251][4:66]
UK_gdp_growth = world_gdp_growth_data.iloc[81][4:66]
x3 = np.arange(1960, 2022, 1)
fig3 = go.Figure(go.Scatter(
                    x = x3,
                    y = world_gdp_growth, 
                    name = 'World GDP growth rate'),layout=go.Layout(title=go.layout.Title(text="GDP growth rate of world main economy"),
                                                             xaxis_title="Year", yaxis_title='GDP growth rate/percent',)
                )
fig3.add_trace(go.Scatter(
                    x = x3,
                    y = China_gdp_growth, 
                    name = 'China GDP growth rate')
                )
fig3.add_trace(go.Scatter(
                    x = x3,
                    y = USA_gdp_growth, 
                    name = 'USA GDP growth rate')
                )
fig3.add_trace(go.Scatter(
                    x = x3,
                    y = UK_gdp_growth, 
                    name = 'UK GDP growth rate')
                )

# show world electricity consumption data
world_electricity_data = pd.read_csv('API_EG.USE.ELEC.KH.PC_DS2_en_csv_v2_4251643.csv')
world_electricity = world_electricity_data.iloc[259][4:66]
China_electricity = world_electricity_data.iloc[40][4:66]
USA_electricity = world_electricity_data.iloc[251][4:66]
UK_electricity = world_electricity_data.iloc[81][4:66]
x4 = np.arange(1960, 2022, 1)
fig4 = go.Figure(go.Scatter(
                    x = x4,
                    y = world_electricity, 
                    name = 'World electricity consumption'),layout=go.Layout(title=go.layout.Title(text="Electricity Consumption of world main economy"),
                                                             xaxis_title="Year", yaxis_title='Electricity consumption / kW·h per capital',)
                )
fig4.add_trace(go.Scatter(
                    x = x4,
                    y = China_electricity, 
                    name = 'China electricity consumption')
                )
fig4.add_trace(go.Scatter(
                    x = x4,
                    y = USA_electricity, 
                    name = 'USA electricity consumption')
                )
fig4.add_trace(go.Scatter(
                    x = x4,
                    y = UK_electricity, 
                    name = 'UK electricity consumption')
                )

# show world population rate data
world_population_data = pd.read_csv('API_SP.POP.GROW_DS2_en_csv_v2_4251293.csv')
world_population = world_population_data.iloc[259][4:66]
China_population = world_population_data.iloc[40][4:66]
USA_population = world_population_data.iloc[251][4:66]
UK_population = world_population_data.iloc[81][4:66]
x5 = np.arange(1960, 2022, 1)
fig5 = go.Figure(go.Scatter(
                    x = x5,
                    y = world_population, 
                    name = 'World population growth rate'),layout=go.Layout(title=go.layout.Title(text="Population growth rate of world main economy"),
                                                             xaxis_title="Year", yaxis_title='Population growth rate / percent',)
                )
fig5.add_trace(go.Scatter(
                    x = x5,
                    y = China_population, 
                    name = 'China population growth rate')
                )
fig5.add_trace(go.Scatter(
                    x = x5,
                    y = USA_population, 
                    name = 'USA population growth rate')
                )
fig5.add_trace(go.Scatter(
                    x = x5,
                    y = UK_population, 
                    name = 'UK population growth rate')
                )

# show world inflation data(GDP deflator)
world_inflation_data = pd.read_csv('API_NY.GDP.DEFL.KD.ZG_DS2_en_csv_v2_4250766.csv')
world_inflation = world_inflation_data.iloc[259][4:66]
China_inflation = world_inflation_data.iloc[40][4:66]
USA_inflation = world_inflation_data.iloc[251][4:66]
UK_inflation = world_inflation_data.iloc[81][4:66]
x6 = np.arange(1960, 2022, 1)
fig6 = go.Figure(go.Scatter(
                    x = x6,
                    y = world_inflation, 
                    name = 'World inflation rate'),layout=go.Layout(title=go.layout.Title(text="inflation rate of world main economy"),
                                                             xaxis_title="Year", yaxis_title='inflation rate / percent',)
                )
fig6.add_trace(go.Scatter(
                    x = x6,
                    y = China_inflation, 
                    name = 'China inflation rate')
                )
fig6.add_trace(go.Scatter(
                    x = x6,
                    y = USA_inflation, 
                    name = 'USA inflation rate')
                )
fig6.add_trace(go.Scatter(
                    x = x6,
                    y = UK_inflation, 
                    name = 'UK inflation rate')
                )

#show gold data
start_date = '2021-1-1'
today = datetime.date.today()
latest_day = today + datetime.timedelta(days=-1)
# end_date = '2022-7-22'
Gold_data = yf.download('GLD', start=start_date, end=latest_day, group_by="ticker", proxy="127.0.0.1:33210",
                                index_col=0, auto_adjust=True)
fig7 = go.Figure(go.Scatter(Gold_data['Close']),layout=go.Layout(title=go.layout.Title(text=f'Gold Price from{start_date} until {today}'),
                                                             xaxis_title="Year", yaxis_title='Gold Price/dollar',)
                )

#show oil data
Oil_data = yf.download('USO', start=start_date, end=today, group_by="ticker", proxy="127.0.0.1:33210",
                               index_col=0, auto_adjust=True)
fig8 = go.Figure(go.Scatter(Oil_data['Close']),layout=go.Layout(title=go.layout.Title(text=f'Oil Price from{start_date} until {today}'),
                                                             xaxis_title="Year", yaxis_title='Oil Price/dollar',)
                )

#show gas data
Gas_data = yf.download('UNG', start=start_date, end=today, group_by="ticker", proxy="127.0.0.1:33210",
                               index_col=0, auto_adjust=True)
fig9 = go.Figure(go.Scatter(asOil_data['Close']),layout=go.Layout(title=go.layout.Title(text=f'Gas Price from{start_date} until {today}'),
                                                             xaxis_title="Year", yaxis_title='Gas Price/dollar',)
                )


In [ ]:
#换页
import dash
import dash_html_components as html
import dash_bootstrap_components as dbc

from dash.dependencies import Input, Output
import dash_core_components as dcc 

# from pandas_datareader import data as web 
import yfinance as yf
from datetime import datetime as Dt

import numpy as np 
import pandas as pd
from ta.trend import MACD
from ta.momentum import StochasticOscillator
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math

import sys
from scipy import stats
import statsmodels.regression.linear_model as lm
import statsmodels.api as sm
import datetime

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# app = dash.Dash(__name__)

app.layout = html.Div(
    dbc.Container(
        dbc.Tabs(
            [
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('The latest international macro-enocomic data are shown below'),
                        dcc.Graph(id='page1-graph1'),
                        dcc.Graph(id='page1-graph2'),
                        dcc.Graph(id='page1-graph3'),
                        dcc.Graph(id='page1-graph4'),
                        dcc.Graph(id='page1-graph5'),
                        dcc.Graph(id='page1-graph6'),
                        dcc.Graph(id='page1-graph7'),
                        dcc.Graph(id='page1-graph8'),
                        dcc.Graph(id='page1-graph9')
                    ],
                    label='Financial Data Dashboard'
                ),
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('Choose the interested stock in the dropdown'),
                        dcc.Dropdown(
                        id='page2-dropdown',
                        options=[
                            {'label': 'Coke', 'value': 'COKE'},
                            {'label': 'Tesla', 'value': 'TSLA'},
                            {'label': 'Apple', 'value': 'AAPL'},
                            {'label': 'Amazon', 'value': 'AMZN'},
                            {'label': 'S&P 500', 'value': '^GSPC'}
                        ],value='TSLA'),  
                        dcc.Graph(id='page2-graph1'),
                        dcc.Graph(id='page2-graph2'),
                        dcc.Graph(id='page2-graph3'),
                        dcc.Graph(id='page2-graph4'),
                        dcc.Graph(id='page2-graph5'),
                        dcc.Graph(id='page2-graph6')
                    ],
                    label='Stock Data Visualization'
                ),
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('Welcome to use CAPM model visualization tool'),
                        html.P('Please select the benchmark'),
                        dcc.Dropdown(
                            id='page3-dropdown2',
                            options=[
                            {'label': 'S&P_500', 'value': '^GSPC'},
                            {'label': 'Nasdaq', 'value': 'NDAQ'},
                            {'label': 'Dow_Jones', 'value': '^DJI'}
                            ],  
                        value='^GSPC'
                        ),
                        dcc.Graph(id='page3-graph1'),
                        dcc.Graph(id='page3-graph2')
                    ],
                    label='CAPM model on selected stock'
                ),
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('Welcome to use Fama French and Q5-factor multi factor model visualization tool'),
                        dcc.Graph(id='page4-graph1'),
                        dcc.Graph(id='page4-graph2'),
                        dcc.Graph(id='page4-graph3'),
                        dcc.Graph(id='page4-graph4'),
                        
                    ],
                    label='Fama French and Q5-factor on selected stock'
                    
                ),
            ]
        ),
        style={'margin-top': '50px'}
    )
)
@app.callback([Output('page2-graph1','figure'),Output('page2-graph2','figure'),Output('page2-graph3','figure'),Output('page2-graph4','figure'),Output('page2-graph5','figure'),Output('page2-graph6','figure'),
              Output('page1-graph1','figure'),Output('page1-graph2','figure'),Output('page1-graph3','figure'),Output('page1-graph4','figure'),Output('page1-graph5','figure'),Output('page1-graph6','figure')
              ,Output('page1-graph7','figure'),Output('page1-graph8','figure'),Output('page1-graph9','figure'),
              Output('page3-graph1','figure'),Output('page3-graph2','figure'),
              Output('page4-graph1','figure'),Output('page4-graph2','figure'),Output('page4-graph3','figure'),
               Output('page4-graph4','figure')],[Input('page2-dropdown', 'value')])
def update_graph(selected_dropdown_value):

    df =  yf.download(selected_dropdown_value, start='2020-01-01', proxy="127.0.0.1:33210")
    stockObject = df.copy(deep=True) # for page3
    
    fig1 = go.Figure(go.Candlestick(x=df.index,
      open=df['Open'],
      high=df['High'],
      low=df['Low'],
      close=df['Close'],
      showlegend=False),
      layout=go.Layout(title=go.layout.Title(text=f"Stock Price and Moving Average of {selected_dropdown_value}")))

    #把非交易日排除掉
    # hide weekends
    fig1.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])]) 
    # removing all empty dates
    # build complete timeline from start date to end date
    dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
    # retrieve the dates that ARE in the original datset
    dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
    # define dates with missing values
    dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
    fig1.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

    #add moving average
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    fig1.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='MA 5'))

    fig1.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='MA 20'))
    fig1.add_trace(go.Scatter(x=df.index, 
                         y=df['MA50'], 
                         opacity=0.7, 
                         line=dict(color='pink', width=2), 
                         name='MA 50'))

    # Plot volume trace on 2nd row 
    fig2 = go.Figure(go.Bar(x=df.index, 
                     y=df['Volume'],
                    ),layout=go.Layout(title=go.layout.Title(text=f"Stock Trading Volumn of {selected_dropdown_value}")))
    # Plot MACD trace on 3rd row
    # MACD
    macd = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)
    fig3 = go.Figure(go.Bar(x=df.index, 
                     y=macd.macd_diff(),
                     name = 'MACD_DIF',
                    ),layout=go.Layout(title=go.layout.Title(text=f"MACD Indicator {selected_dropdown_value}")))
    fig3.add_trace(go.Scatter(x=df.index,
                         y=macd.macd(),
                         line=dict(color='black', width=2),
                         name = 'MACD_DEA'
                        ))
    fig3.add_trace(go.Scatter(x=df.index,
                         y=macd.macd_signal(),
                         line=dict(color='blue', width=1),
                         name = 'MACD moving average of 3 days'
                        ))
    # Plot stochastics trace on 4th row
    # stochastic
    stoch = StochasticOscillator(high=df['High'],
                             close=df['Close'],
                             low=df['Low'],
                             window=14, 
                             smooth_window=3)
    fig4 = go.Figure(go.Scatter(x=df.index,
                         y=stoch.stoch(),
                         line=dict(color='black', width=2),
                         name = 'stochastic'
                        ),layout=go.Layout(title=go.layout.Title(text=f"Stochastic of {selected_dropdown_value}")))
    fig4.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch_signal(),
                         name = 'stochastic moving average of 3 days',
                         line=dict(color='blue', width=1),
                        ))
    #传统的stochastics oscillator 80超买，20超卖,把这个加进stochastics
    
    const1 = np.linspace(start=20, stop=20,num=len(df.index))
    const2 = np.linspace(start=80, stop=80,num=len(df.index))
    fig4.add_trace(go.Scatter(x=df.index,
                         y=const1,
                         line=dict(color='red',width=1),
                         name = '20'
                        ))
    fig4.add_trace(go.Scatter(x=df.index,
                         y=const2,
                         line=dict(color='green',width=1),
                         name = '80'
                        ))
    
    #fig5 volatility
    df.insert(df.shape[1],'Ln',0)
    df.insert(df.shape[1],'Ln2',0)

    for i in range(df.shape[0]-1):
        df['Ln'].iloc[i+1] = math.log(df['Close'].iloc[i+1]/df['Close'].iloc[i])
        df['Ln2'].iloc[i+1] = df['Ln'].iloc[i+1]**2
    df.insert(df.shape[1],'Volatility',0)
    for i in range(df.shape[0]-2):
        df['Volatility'].iloc[i+2] = math.sqrt(df['Ln2'].iloc[1:i+2].sum()/(i+2-1))
    fig5 = go.Figure(go.Scatter(x=df.index,
                         y=df['Volatility'],
                         line=dict(color='blue',width=1)
                        ),layout=go.Layout(title=go.layout.Title(text=f"Volatility of {selected_dropdown_value}")))
    
    #figure6 value at risk
    df['daily_ret'] = df['Close'].pct_change(1)
    # print(df)
    sRate = df['daily_ret'].iloc[1:].sort_values(ascending=True)
    # print(sRate)
    p = np.percentile(sRate, (1, 5, 10), interpolation='midpoint') # 输出分位度为1%，5%和10%即置信度99%，95%和90%时的值
    print(p) #1%分位值为第一个-0.05377872，即根据历史数据，value at risk回报率高于-0.05377872的可能为99%
    fig6 = go.Figure(go.Bar(x=['1%','5%','10%'],
                     y=p
                     ),layout=go.Layout(title=go.layout.Title(text=f"Value at risk on 1%, 5% and 10% of {selected_dropdown_value}")))
    
    #show gdp data
    world_gdp_data = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4251000.csv')
    # world_gdp_data = world_gdp_data.reset_index(drop=True)
    world_gdp = world_gdp_data.iloc[259][4:66]
    China_gdp = world_gdp_data.iloc[40][4:66]
    USA_gdp = world_gdp_data.iloc[251][4:66]
    UK_gdp = world_gdp_data.iloc[81][4:66]
        # print(Russian_gdp)
    x = np.arange(1960, 2022, 1)
    page1_fig1 = go.Figure(go.Scatter(
                    x = x,
                    y = world_gdp, 
                    name = 'World GDP data'),layout=go.Layout(title=go.layout.Title(text="GDP data of world main economy"),
                                                             xaxis_title="Year", yaxis_title="GDP/million dollars",)
                )
    page1_fig1.add_trace(go.Scatter(
                    x = x,
                    y = China_gdp, 
                    name = 'China GDP data')
                )
    page1_fig1.add_trace(go.Scatter(
                    x = x,
                    y = USA_gdp, 
                    name = 'USA GDP data')
                )
    page1_fig1.add_trace(go.Scatter(
                    x = x,
                    y = UK_gdp, 
                    name = 'UK GDP data')
                )

    #show gdp per capital data
    world_gdp_perCapital_data = pd.read_csv('API_NY.GDP.PCAP.CD_DS2_en_csv_v2_4251004.csv')
    world_gdp_perCapital = world_gdp_perCapital_data.iloc[259][4:66]
    China_gdp_perCapital = world_gdp_perCapital_data.iloc[40][4:66]
    USA_gdp_perCapital = world_gdp_perCapital_data.iloc[251][4:66]
    UK_gdp_perCapital = world_gdp_perCapital_data.iloc[81][4:66]
    x2 = np.arange(1960, 2022, 1)
    page1_fig2 = go.Figure(go.Scatter(
                    x = x2,
                    y = world_gdp_perCapital, 
                    name = 'World GDP per capital data'),layout=go.Layout(title=go.layout.Title(text="GDP per capital data of world main economy"),
                                                             xaxis_title="Year", yaxis_title='GDP per capital/dollars',)
                )
    page1_fig2.add_trace(go.Scatter(
                    x = x2,
                    y = China_gdp_perCapital, 
                    name = 'China GDP per capital data')
                )
    page1_fig2.add_trace(go.Scatter(
                    x = x2,
                    y = USA_gdp_perCapital, 
                    name = 'USA GDP per capital data')
                )
    page1_fig2.add_trace(go.Scatter(
                    x = x2,
                    y = UK_gdp_perCapital, 
                    name = 'UK GDP per capitaldata')
                )

    #show gdp growth rate data
    world_gdp_growth_data = pd.read_csv('API_NY.GDP.PCAP.KD.ZG_DS2_en_csv_v2_4250851.csv')
    world_gdp_growth = world_gdp_growth_data.iloc[259][4:66]
    China_gdp_growth = world_gdp_growth_data.iloc[40][4:66]
    USA_gdp_growth = world_gdp_growth_data.iloc[251][4:66]
    UK_gdp_growth = world_gdp_growth_data.iloc[81][4:66]
    x3 = np.arange(1960, 2022, 1)
    page1_fig3 = go.Figure(go.Scatter(
                    x = x3,
                    y = world_gdp_growth, 
                    name = 'World GDP growth rate'),layout=go.Layout(title=go.layout.Title(text="GDP growth rate of world main economy"),
                                                             xaxis_title="Year", yaxis_title='GDP growth rate/percent',)
                )
    page1_fig3.add_trace(go.Scatter(
                    x = x3,
                    y = China_gdp_growth, 
                    name = 'China GDP growth rate')
                )
    page1_fig3.add_trace(go.Scatter(
                    x = x3,
                    y = USA_gdp_growth, 
                    name = 'USA GDP growth rate')
                )
    page1_fig3.add_trace(go.Scatter(
                    x = x3,
                    y = UK_gdp_growth, 
                    name = 'UK GDP growth rate')
                )

    # show world electricity consumption data
    world_electricity_data = pd.read_csv('API_EG.USE.ELEC.KH.PC_DS2_en_csv_v2_4251643.csv')
    world_electricity = world_electricity_data.iloc[259][4:66]
    China_electricity = world_electricity_data.iloc[40][4:66]
    USA_electricity = world_electricity_data.iloc[251][4:66]
    UK_electricity = world_electricity_data.iloc[81][4:66]
    x4 = np.arange(1960, 2022, 1)
    page1_fig4 = go.Figure(go.Scatter(
                    x = x4,
                    y = world_electricity, 
                    name = 'World electricity consumption'),layout=go.Layout(title=go.layout.Title(text="Electricity Consumption of world main economy"),
                                                             xaxis_title="Year", yaxis_title='Electricity consumption / kW·h per capital',)
                )
    page1_fig4.add_trace(go.Scatter(
                    x = x4,
                    y = China_electricity, 
                    name = 'China electricity consumption')
                )
    page1_fig4.add_trace(go.Scatter(
                    x = x4,
                    y = USA_electricity, 
                    name = 'USA electricity consumption')
                )
    page1_fig4.add_trace(go.Scatter(
                    x = x4,
                    y = UK_electricity, 
                    name = 'UK electricity consumption')
                )

    # show world population rate data
    world_population_data = pd.read_csv('API_SP.POP.GROW_DS2_en_csv_v2_4251293.csv')
    world_population = world_population_data.iloc[259][4:66]
    China_population = world_population_data.iloc[40][4:66]
    USA_population = world_population_data.iloc[251][4:66]
    UK_population = world_population_data.iloc[81][4:66]
    x5 = np.arange(1960, 2022, 1)
    page1_fig5 = go.Figure(go.Scatter(
                    x = x5,
                    y = world_population, 
                    name = 'World population growth rate'),layout=go.Layout(title=go.layout.Title(text="Population growth rate of world main economy"),
                                                             xaxis_title="Year", yaxis_title='Population growth rate / percent',)
                )
    page1_fig5.add_trace(go.Scatter(
                    x = x5,
                    y = China_population, 
                    name = 'China population growth rate')
                )
    page1_fig5.add_trace(go.Scatter(
                    x = x5,
                    y = USA_population, 
                    name = 'USA population growth rate')
                )
    page1_fig5.add_trace(go.Scatter(
                    x = x5,
                    y = UK_population, 
                    name = 'UK population growth rate')
                )

    # show world inflation data(GDP deflator)
    world_inflation_data = pd.read_csv('API_NY.GDP.DEFL.KD.ZG_DS2_en_csv_v2_4250766.csv')
    world_inflation = world_inflation_data.iloc[259][4:66]
    China_inflation = world_inflation_data.iloc[40][4:66]
    USA_inflation = world_inflation_data.iloc[251][4:66]
    UK_inflation = world_inflation_data.iloc[81][4:66]
    x6 = np.arange(1960, 2022, 1)
    page1_fig6 = go.Figure(go.Scatter(
                    x = x6,
                    y = world_inflation, 
                    name = 'World inflation rate'),layout=go.Layout(title=go.layout.Title(text="inflation rate of world main economy"),
                                                             xaxis_title="Year", yaxis_title='inflation rate / percent',)
                )
    page1_fig6.add_trace(go.Scatter(
                    x = x6,
                    y = China_inflation, 
                    name = 'China inflation rate')
                )
    page1_fig6.add_trace(go.Scatter(
                    x = x6,
                    y = USA_inflation, 
                    name = 'USA inflation rate')
                )
    page1_fig6.add_trace(go.Scatter(
                    x = x6,
                    y = UK_inflation, 
                    name = 'UK inflation rate')
                )

    #show gold data
    start_date = '2021-1-1'
    today = datetime.date.today()
    latest_day = today + datetime.timedelta(days=-1)
    # end_date = '2022-7-22'
    Gold_data = yf.download('GLD', start=start_date, end=latest_day, group_by="ticker", proxy="127.0.0.1:33210",
                                index_col=0, auto_adjust=True)
    page1_fig7 = go.Figure(go.Scatter(
                x = Gold_data.index, y=Gold_data['Close']),layout=go.Layout(title=go.layout.Title(text=f'Gold Price from{start_date} until {today}'),
                                                             xaxis_title="Year", yaxis_title='Gold Price/dollar',)
                )

    #show oil data
    Oil_data = yf.download('USO', start=start_date, end=today, group_by="ticker", proxy="127.0.0.1:33210",
                               index_col=0, auto_adjust=True)
    page1_fig8 = go.Figure(go.Scatter(
                x = Oil_data.index, y=Oil_data['Close']),layout=go.Layout(title=go.layout.Title(text=f'Oil Price from{start_date} until {today}'),
                                                             xaxis_title="Year", yaxis_title='Oil Price/dollar',)
                )

    #show gas data
    Gas_data = yf.download('UNG', start=start_date, end=today, group_by="ticker", proxy="127.0.0.1:33210",
                               index_col=0, auto_adjust=True)
    page1_fig9 = go.Figure(go.Scatter(
                x = Gas_data.index,y=Gas_data['Close']),layout=go.Layout(title=go.layout.Title(text=f'Gas Price from{start_date} until {today}'),
                                                             xaxis_title="Year", yaxis_title='Gas Price/dollar',)
                )

#     page3 capm model
    RISKY_ASSET = selected_dropdown_value
    MARKET_BENCHMARK = '^GSPC'
#     stockObject_page3 = yf.download(RISKY_ASSET, start='2020-01-01', group_by="ticker",
#                                   proxy="127.0.0.1:33210")  # acquire data
#     stockObject = df.copy(deep=True)
    marketBenchmark = yf.download(MARKET_BENCHMARK, start='2020-01-01', group_by="ticker",
                                      proxy="127.0.0.1:33210")

    stockObject['Close'] = stockObject['Close'] / stockObject['Close'].iloc[0]
    marketBenchmark['Close'] = marketBenchmark['Close'] / marketBenchmark['Close'].iloc[0]
    stockObject['daily_ret'] = stockObject['Close'].pct_change(1)
    marketBenchmark['daily_ret'] = marketBenchmark['Close'].pct_change(1)
    lr = stats.linregress(stockObject['daily_ret'].iloc[1:], marketBenchmark['daily_ret'].iloc[1:])

    beta, alpha, r_value, p_value, std_err = lr
    rf = 1.2855 / 100  # UK 1-Year Treasury Bond on 30/03/2022
    Ef = (marketBenchmark['Close'].iloc[marketBenchmark['Close'].shape[0] - 1] - marketBenchmark['Close'].iloc[0]) / \
             marketBenchmark['Close'].iloc[0]
    CAPM_expectReturnRate = rf + beta * (Ef - rf)
    page3_fig1 = go.Figure(go.Bar(
                            x=['risk-free rate', 'market rate', f'CAPM model rate of {RISKY_ASSET}'],
                            y=[rf, Ef, CAPM_expectReturnRate]),layout=go.Layout(title=go.layout.Title(text=f'Investing Return Rate of Risk-free Rate, Market Rate and CAPM model Rate of {RISKY_ASSET}'),
                                                             xaxis_title="investing rate comparison", yaxis_title='investing return rate')
                          )
    
    RiskFreeRate = 1.2855 / 100  # UK 1-Year Treasury Bond on 30/03/2022
    sharpe_stock = (stockObject['Close'] - RiskFreeRate).mean() / (stockObject['Close'] - RiskFreeRate).std()
    sharpe_market = (marketBenchmark['Close'] - RiskFreeRate).mean() / (marketBenchmark['Close'] - RiskFreeRate).std()
    page3_fig2 = go.Figure(go.Bar(
                            x=[f'Sharpe Ratio of {RISKY_ASSET}', f'Sharpe Ratio of {MARKET_BENCHMARK}'],
                            y=[sharpe_stock, sharpe_market]),layout=go.Layout(title=go.layout.Title(text=f'Sharpe Ratio of {RISKY_ASSET} and Market Benchmark'),
                                                             xaxis_title="Sharpe Ratio Comparison", yaxis_title='Sharpe Ratio')
                          )
    
    #page4
    page4_start_date = '2019-1-1'
    page4_end_date = '2021-12-30'
    stockObject_page4 = yf.download(selected_dropdown_value, start=page4_start_date, end=page4_end_date, group_by="ticker",proxy="127.0.0.1:33210", index_col=0)
        # stockObject_qFactor = yf.download(RISKY_ASSET, start=START_DATE, end=END_DATE, group_by="ticker",
        #                           proxy="127.0.0.1:33210", index_col=0)
    stockObject_page4_qFactor = stockObject_page4.copy(deep=True)# 复制一份给qfactor用
    returns_3 = pd.read_csv('F-F_Research_Data_Factors_daily.csv', index_col=0)
    returns_5 = pd.read_csv('F-F_Research_Data_5_Factors_2x3_daily.csv', index_col=0)
    returns_3['Date'] = returns_3.index
    returns_5['Date'] = returns_5.index
    stockObject_page4['daily_ret'] = stockObject_page4['Close'].pct_change(1)

    # 观察发现3因子和5因子数据集的SMB不一样，Mkt-RF和HML是一样的
    stockObject_page4['Date'] = stockObject_page4.index
    stockObject_page4.insert(stockObject_page4.shape[1], 'Mkt-RF', 0)
    stockObject_page4.insert(stockObject_page4.shape[1], 'SMB_3', 0)
    stockObject_page4.insert(stockObject_page4.shape[1], 'SMB_5', 0)
    stockObject_page4.insert(stockObject_page4.shape[1], 'HML', 0)
    stockObject_page4.insert(stockObject_page4.shape[1], 'RMW', 0)
    stockObject_page4.insert(stockObject_page4.shape[1], 'CMA', 0)
    stockObject_page4.insert(stockObject_page4.shape[1], 'RF', 0)

    # 合并fama french五因子数据到yf 股票data里 yf下载的数据可能比上面得到的end_iloc要提前（yf似乎下载指定结束日期之前交易日的数据）
    for i in range(returns_3.shape[0]):
        if Dt.strptime(returns_3['Date'][i], '%Y/%m/%d') == Dt.strptime(str(stockObject_page4['Date'].iloc[0]).split(' ')[0], '%Y-%m-%d'):
            iloc_offset_3 = i
#                 print('OFFSET iteration is', iloc_offset_3)
    for i in range(returns_5.shape[0]):
        if Dt.strptime(returns_5['Date'][i], '%Y/%m/%d') == Dt.strptime(str(stockObject_page4['Date'].iloc[0]).split(' ')[0], '%Y-%m-%d'):
            iloc_offset_5 = i
#                 print('OFFSET iteration is', iloc_offset_5)
        # 然后，把stockObject那段长度之前的fama-french数据放进stockObject

    for i in range(stockObject.shape[0]):
        stockObject_page4['Mkt-RF'].iloc[i] = returns_3['Mkt-RF'].iloc[i + iloc_offset_3]
        stockObject_page4['SMB_3'].iloc[i] = returns_3['SMB'].iloc[i + iloc_offset_3]
        stockObject_page4['SMB_5'].iloc[i] = returns_5['SMB'].iloc[i + iloc_offset_5]
        stockObject_page4['HML'].iloc[i] = returns_3['HML'].iloc[i + iloc_offset_3]
        stockObject_page4['RMW'].iloc[i] = returns_5['RMW'].iloc[i + iloc_offset_5]
        stockObject_page4['CMA'].iloc[i] = returns_5['CMA'].iloc[i + iloc_offset_5]
        stockObject_page4['RF'].iloc[i] = returns_3['RF'].iloc[i + iloc_offset_3]
        stockObject_page4['daily_ret'].iloc[i] -= returns_3['RF'].iloc[i + iloc_offset_3]

    # stockObject.loc[1:,'CT'] = ct.add_constant(stockObject) #ct是多元线性拟合的截距项
    x_ff3 = stockObject_page4[['Mkt-RF', 'SMB_3', 'HML']].iloc[1:]
    y_ff3 = stockObject_page4['daily_ret'].iloc[1:]
    x_ff3 = sm.add_constant(x_ff3)
    ff3fm = lm.OLS(y_ff3, x_ff3).fit()
    x_ff5 = stockObject_page4[['Mkt-RF', 'SMB_5', 'HML', 'RMW', 'CMA']].iloc[1:]
    y_ff5 = stockObject_page4['daily_ret'].iloc[1:]
    x_ff5 = sm.add_constant(x_ff5)
    ff5fm = lm.OLS(y_ff5, x_ff5).fit()
    print('Fama French 3 factors summary')
    print(ff3fm.summary())
    print('Fama French 5 factors summary')
    print(ff5fm.summary())

    # 绘制ff5fm模型预测收益率，和实际进行可视化对比
    intercept3, beta_Mkt_RF_3, beta_SMB_3, beta_HML_3 = ff3fm.params
    intercept5, beta_Mkt_RF_5, beta_SMB_5, beta_HML_5, beta_RMW, beta_CMA = ff5fm.params
    stockObject_page4.insert(stockObject_page4.shape[1], 'ff3fm_return', 0)
    stockObject_page4.insert(stockObject_page4.shape[1], 'ff5fm_return', 0)
    stockObject_page4['ff3fm_return'] = stockObject_page4['RF'] + beta_Mkt_RF_3 * stockObject_page4['Mkt-RF'] + beta_SMB_3 * stockObject_page4['SMB_3'] + beta_HML_3 * stockObject_page4['HML']
    stockObject_page4['ff5fm_return'] = stockObject_page4['RF'] + beta_Mkt_RF_5 * stockObject_page4['Mkt-RF'] + beta_SMB_5 * stockObject_page4['SMB_5'] + beta_HML_5 * stockObject_page4['HML'] + beta_RMW * stockObject_page4['RMW'] + beta_CMA * stockObject_page4['CMA']

    #跑一下q5模型
    q_returns = pd.read_csv('q5_factors_daily_2021_2.csv', index_col=0)
    q_returns.columns.tolist
    # returns.rename_axis('Date',axis='columns')
    q_returns['Date'] = q_returns.index
    # print(q_returns)
    stockObject_page4_qFactor['daily_ret'] = stockObject_page4_qFactor['Close'].pct_change(1)

    stockObject_page4_qFactor['Date'] = stockObject_page4_qFactor.index
    stockObject_page4_qFactor.insert(stockObject_page4_qFactor.shape[1], 'Mkt-RF', 0)
    stockObject_page4_qFactor.insert(stockObject_page4_qFactor.shape[1], 'ME', 0)
    stockObject_page4_qFactor.insert(stockObject_page4_qFactor.shape[1], 'IA', 0)
    stockObject_page4_qFactor.insert(stockObject_page4_qFactor.shape[1], 'ROE', 0)
    stockObject_page4_qFactor.insert(stockObject_page4_qFactor.shape[1], 'EG', 0)
    stockObject_page4_qFactor.insert(stockObject_page4_qFactor.shape[1], 'RF', 0)

    for i in range(q_returns.shape[0]):
        if Dt.strptime(q_returns['Date'][i], '%Y/%m/%d') == Dt.strptime(str(stockObject_page4_qFactor['Date'].iloc[0]).split(' ')[0], '%Y-%m-%d'):
            iloc_offset_qFactor = i
#         print('qFactor OFFSET iteration is', iloc_offset_qFactor)
    for i in range(stockObject_page4_qFactor.shape[0]):
        stockObject_page4_qFactor['Mkt-RF'].iloc[i] = q_returns['R_MKT'].iloc[i + iloc_offset_qFactor] - q_returns['R_F'].iloc[i + iloc_offset_qFactor]
        stockObject_page4_qFactor['ME'].iloc[i] = q_returns['R_ME'].iloc[i + iloc_offset_qFactor]
        stockObject_page4_qFactor['IA'].iloc[i] = q_returns['R_IA'].iloc[i + iloc_offset_qFactor]
        stockObject_page4_qFactor['ROE'].iloc[i] = q_returns['R_ROE'].iloc[i + iloc_offset_qFactor]
        stockObject_page4_qFactor['EG'].iloc[i] = q_returns['R_EG'].iloc[i + iloc_offset_qFactor]
        stockObject_page4_qFactor['RF'].iloc[i] = q_returns['R_F'].iloc[i + iloc_offset_qFactor]
        stockObject_page4_qFactor['daily_ret'].iloc[i] -= q_returns['R_F'].iloc[i + iloc_offset_qFactor]
    # stockObject_qFactor.to_csv('stockObject.csv')
    x_qf = stockObject_page4_qFactor[['Mkt-RF', 'ME', 'IA', 'ROE', 'EG']].iloc[1:]
    y_qf = stockObject_page4_qFactor['daily_ret'].iloc[1:]
    x_qf = sm.add_constant(x_qf)
    Qf = lm.OLS(y_qf, x_qf).fit()
    print('q5 factors summary')
    print(Qf.summary())
    intercept_q5, beta_Mkt_RF_q5, beta_ME, beta_IA, beta_ROE, beta_EG = Qf.params
    stockObject_page4_qFactor.insert(stockObject_page4_qFactor.shape[1], 'qf_return', 0)
    stockObject_page4_qFactor['qf_return'] = intercept_q5 + stockObject_page4_qFactor['RF'] + beta_Mkt_RF_q5 * stockObject_page4_qFactor['Mkt-RF'] + beta_ME * stockObject_page4_qFactor['ME'] + beta_IA * stockObject_page4_qFactor['IA'] + beta_ROE * stockObject_page4_qFactor['ROE'] + beta_EG * stockObject_page4_qFactor['EG']

    #显示ff3、5和q5 收益率与真实收益率
    page4_fig1 = go.Figure(go.Scatter(
                    x = stockObject_page4.index,
                    y = stockObject['daily_ret'],
                    name = 'Actual daily return'),layout=go.Layout(title=go.layout.Title(text=f'Comparison of Actual daily return, Fama French and q5 factor return of {RISKY_ASSET}'),
                                                             xaxis_title='Date', yaxis_title='Return Rate')
                          )
    page4_fig1.add_trace(go.Scatter(
                    x = stockObject_page4.index,
                    y = stockObject_page4['ff3fm_return'],
                    name = 'Fama French 3 factor model daily return')
                        )
    page4_fig1.add_trace(go.Scatter(
                    x = stockObject_page4.index,
                    y = stockObject_page4['ff5fm_return'],
                    name = 'Fama French 5 factor model daily return')
                        )
    page4_fig2 = go.Figure(go.Bar(
                    x = ['beta_Mkt-RF', 'beta_SMB', 'beta_HML', 'alpha'],
                    y = [beta_Mkt_RF_3, beta_SMB_3, beta_HML_3, intercept3]),
                           layout=go.Layout(title=go.layout.Title(text=f'Beta value of coefficient of Fama French 3 factor of {RISKY_ASSET}'),
                                                             xaxis_title='Coefficient value of Fama French Factors', yaxis_title='Coefficient value')
                          )
    
    page4_fig3 = go.Figure(go.Bar(
                    x = ['beta_Mkt-RF', 'beta_SMB', 'beta_HML', 'beta_RMW', 'beta_CMA', 'alpha'],
                    y = [beta_Mkt_RF_5, beta_SMB_5, beta_HML_5, beta_RMW, beta_CMA, intercept5]),
                           layout=go.Layout(title=go.layout.Title(text=f'Beta value of coefficient of Fama French 5 factor of {RISKY_ASSET}'),
                                                             xaxis_title='Coefficient value of Fama French Factors', yaxis_title='Coefficient value')
                          )
    page4_fig4 = go.Figure(go.Bar(
                    x = ['beta_Mkt-RF', 'beta_ME', 'beta_IA', 'beta_ROE', 'beta_EG', 'alpha'],
                    y = [beta_Mkt_RF_q5, beta_ME, beta_IA, beta_ROE, beta_EG, intercept_q5]),
                           layout=go.Layout(title=go.layout.Title(text=f'Beta value of coefficient of q5 factor of {RISKY_ASSET}'),
                                                             xaxis_title='Coefficient value of q5 Factors', yaxis_title='Coefficient value')
                          )
                    
    return fig1, fig2, fig3, fig4, fig5, fig6, page1_fig1, page1_fig2, page1_fig3, page1_fig4, page1_fig5, page1_fig6, page1_fig7, page1_fig8, page1_fig9, page3_fig1, page3_fig2, page4_fig1, page4_fig2, page4_fig3, page4_fig4

# # page1
# @app.callback([Output('page1-graph1','figure'),Output('page1-graph2','figure'),Output('page1-graph3','figure'),Output('page1-graph4','figure'),Output('page1-graph5','figure'),Output('page2-graph6','figure'),Output('page1-graph7','figure'),Output('page1-graph8','figure'),Output('page2-graph9','figure')], [Input('page2-dropdown', 'value')])
# def update_graph():
    


if __name__ == '__main__':
    app.run_server(port=8051)
# if __name__ == '__main__':
#     app.run_server(debug=True)

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is run

 * Running on http://127.0.0.1:8051 (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2022 16:08:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 16:08:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 16:08:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 16:08:19] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Jul/2022 16:08:19] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Jul/2022 16:08:19] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


[*********************100%***********************]  1 of 1 completed


C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:222: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:223: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:226: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:237: DeprecationWarnin

[-0.12087212 -0.06521483 -0.0470029 ]
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:517: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:518: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:519: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:520: SettingWithCopyWa

Fama French 3 factors summary
                            OLS Regression Results                            
Dep. Variable:              daily_ret   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.271
Method:                 Least Squares   F-statistic:                     94.55
Date:                Thu, 21 Jul 2022   Prob (F-statistic):           6.44e-52
Time:                        16:08:31   Log-Likelihood:                 1430.7
No. Observations:                 755   AIC:                            -2853.
Df Residuals:                     751   BIC:                            -2835.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0005 

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:569: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:570: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:571: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_11348\3956667141.py:572: SettingWithCopyWa

q5 factors summary
                            OLS Regression Results                            
Dep. Variable:              daily_ret   R-squared:                       0.292
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     61.65
Date:                Thu, 21 Jul 2022   Prob (F-statistic):           7.63e-54
Time:                        16:08:40   Log-Likelihood:                 1440.0
No. Observations:                 755   AIC:                            -2868.
Df Residuals:                     749   BIC:                            -2840.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0001      0.001 